In [1]:
from IPython.display import display

from sympy import *

from monom import *

In [2]:
t, x, tau, h, s0, s1, s = symbols(r't, x, tau, h, sigma_0, sigma_1, sigma', real=True)
f1, f2 = (f(t, x) for f in symbols('\phi_1, \phi_2', cls=Function))

In [3]:
pda_f, pda_v = (f1, f2), (t, x)
pda_n, pda_clp = 5, 3
pda_p = Rational(0, 1), Rational(0, 1), Rational(0, 1)

In [4]:
Monom.variables = 1 + len(pda_v)
# Monom.cmp = Monom.TOPdeglex
Monom.cmp = Monom.TOPlex
Monom.zero = Monom(0 for v in range(Monom.variables))
pda_fun = dict(zip(pda_f,\
             (Monom(0 if v else i for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))
pda_var = dict(zip(pda_v,\
             (Monom(0 if v != i else 1 for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))

In [5]:
def T(f, i1, j1):
    return sum(sum(\
        diff(f, t, i, x, j)*(tau*(i1+pda_p[0]))**i \
                             *(h*(j1+pda_p[1]))**j/\
                 (factorial(i)*factorial(j))\
        for i in range(pda_n-j))\
        for j in range(pda_n))

In [6]:
expand((T(f1, 0, 1)-T(f1, 0, -1))/(2*h))

h**2*Derivative(\phi_1(t, x), (x, 3))/6 + Derivative(\phi_1(t, x), x)

In [9]:
def clip(f):
    f = f.expand()
    return [f.coeff(tau, 0).coeff(h, 0),\
            f.coeff(h, 0).coeff(tau, 2),\
            f.coeff(tau, 0).coeff(h, 2)]

def df2m(a):
    assert a.func == Derivative
    m = pda_fun[a.args[0]]
    for xi in a.args[1:]:
        if isinstance(xi, Symbol):
            m = m*pda_var[xi]
        else:
            m = m*pda_var[xi[0]]**xi[1]
    return m

def m2df(m):
    r = pda_f[m[0]-1]
    for i in range(1, len(m)):
        r = r.diff(pda_v[i-1], m[i])
    return r

def findDiv(a, d):
    r = None
    def find(a, r):
        if a.args:
            if a.func == Derivative and a.args[0] in pda_fun:
                m = df2m(a)
                if m.divisible(d) and (not r or m.cmp(r) > 0):
                    r = m
            else:
                for s in a.args:
                    r = find(s, r)
        return r
    return find(a, r)

def reduction(f1, f2, m, c, shift):
    assert shift < pda_clp
    r = [f1[i] for i in range(shift)]
    if not m:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift]*c))
    else:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift].diff(*m)*c))
    return r

def NF(f, df, G, head=False, trace=False):
    assert len(df) == len(G)
#     print(df2m(df[0]))
    ms = [df2m(d) for d in df]
    for i in range(0 if head else 1, pda_clp):
        t = 0
        if f[i]:
            while True:
                r = None
                for l in range(len(ms)):
                    r = findDiv(f[i], ms[l])
                    if r:
                        break
                if not r: 
                    break
                c, deg, m = 0, 7, m2df(r)
                while c == 0:
                    c = f[i].coeff(m, deg)
                    deg -= 1
                    assert deg >= 0
#                 print(c, m, deg+1)
                if deg:
                    c *= m**deg
                m = r/ms[l]
                d = []
                for k in range(len(pda_v)):
                    if m[k+1] > 0:
                        d.append(pda_v[k])
                        if m[k+1] > 1:
                            d.append(m[k+1])
                if trace:
                    print(">"*12)
                    eq = Symbol("eq%d" % (l+1), real=True)
                    if d:
                        display(Derivative(*tuple([eq] + d))*c*h**i)
                    else:
                        display(eq*c*h**i)
                f = reduction(f, G[l], tuple(d), c/G[l][0].coeff(df[l]), i)
                if trace:
                    print("res =")
                    display(f)
                    print("<"*12)
                t += 1
#                 if t > 6: break
    return f

def p2Latex(a):
    r = None
    def find(a):
        if not a.args:
            return a
        elif a in pda_fun:
            return Symbol(str(a.func), real=True)
        else:
            if a.func == Derivative and a.args[0] in pda_fun:
                m, r = df2m(a), [str(a.args[0].func)[-1]]
                for v in range(1, Monom.variables):
                    for i in range(m[v]):
                        r.append(str(pda_v[v-1]))
                return Symbol(str(a.args[0].func)[:-2] + "_{%s}" % "".join(r), real=True)
            else:
                return a.func(*(find(s) for s in a.args))
    return find(a)

def prn(a):
    display(p2Latex(a[0]))
    print("tau^2 =>")
    display(p2Latex(a[1]))
    print("h^2 =>")
    display(p2Latex(a[2]))

In [10]:
def Dt(a):
    return (T(a, 1, 0) - T(a, 0, 0))/(tau)
def Dx(a):
    return (T(a, 0, 1) - T(a, 0, -1))/(2*h)
def Dxxx(a):
    return (T(a, 0, 2) - 2*T(a, 0, 1) + 2*T(a, 0, -1) - T(a, 0, -2))/(2*h**3)
def It(a):
    return (T(a, 1, 0) + T(a, 0, 0))/2


In [11]:
F1 = clip(Dt(f1) + It(3*s0*Dx(f1**2) + 2*s1*Dx(f1**3) + Dxxx(f1) + f1 - f2))
prn(F1)

6*\phi_1**2*\phi_{1x}*sigma_1 + 6*\phi_1*\phi_{1x}*sigma_0 + \phi_1 - \phi_2 + \phi_{1t} + \phi_{1xxx}

tau^2 =>


3*\phi_1**2*\phi_{1ttx}*sigma_1/2 + 3*\phi_1*\phi_{1ttx}*sigma_0/2 + 3*\phi_1*\phi_{1tt}*\phi_{1x}*sigma_1 + 6*\phi_1*\phi_{1tx}*\phi_{1t}*sigma_1 + \phi_{1ttt}/6 + \phi_{1ttxxx}/4 + 3*\phi_{1tt}*\phi_{1x}*sigma_0/2 + \phi_{1tt}/4 + 3*\phi_{1tx}*\phi_{1t}*sigma_0 + 3*\phi_{1t}**2*\phi_{1x}*sigma_1 - \phi_{2tt}/4

h^2 =>


\phi_1**2*\phi_{1xxx}*sigma_1 + \phi_1*\phi_{1xxx}*sigma_0 + 6*\phi_1*\phi_{1xx}*\phi_{1x}*sigma_1 + 3*\phi_{1xx}*\phi_{1x}*sigma_0 + 2*\phi_{1x}**3*sigma_1

In [13]:
F2 = clip(Dt(f2) + It(3*s0*Dx(f2**2) + 2*s1*Dx(f2**3) + Dxxx(f2) + f2 - f1 + s*f2))
prn(F2)

-\phi_1 + 6*\phi_2**2*\phi_{2x}*sigma_1 + 6*\phi_2*\phi_{2x}*sigma_0 + \phi_2*sigma + \phi_2 + \phi_{2t} + \phi_{2xxx}

tau^2 =>


3*\phi_2**2*\phi_{2ttx}*sigma_1/2 + 3*\phi_2*\phi_{2ttx}*sigma_0/2 + 3*\phi_2*\phi_{2tt}*\phi_{2x}*sigma_1 + 6*\phi_2*\phi_{2tx}*\phi_{2t}*sigma_1 - \phi_{1tt}/4 + \phi_{2ttt}/6 + \phi_{2ttxxx}/4 + 3*\phi_{2tt}*\phi_{2x}*sigma_0/2 + \phi_{2tt}*sigma/4 + \phi_{2tt}/4 + 3*\phi_{2tx}*\phi_{2t}*sigma_0 + 3*\phi_{2t}**2*\phi_{2x}*sigma_1

h^2 =>


\phi_2**2*\phi_{2xxx}*sigma_1 + \phi_2*\phi_{2xxx}*sigma_0 + 6*\phi_2*\phi_{2xx}*\phi_{2x}*sigma_1 + 3*\phi_{2xx}*\phi_{2x}*sigma_0 + 2*\phi_{2x}**3*sigma_1

In [14]:
F1pda = NF(F1, [f1.diff(t), f2.diff(t)], [F1, F2], head=False)
prn(F1pda)

6*\phi_1**2*\phi_{1x}*sigma_1 + 6*\phi_1*\phi_{1x}*sigma_0 + \phi_1 - \phi_2 + \phi_{1t} + \phi_{1xxx}

tau^2 =>


18*\phi_1**6*\phi_{1xxx}*sigma_1**3 + 54*\phi_1**5*\phi_{1xxx}*sigma_0*sigma_1**2 + 324*\phi_1**5*\phi_{1xx}*\phi_{1x}*sigma_1**3 + 9*\phi_1**4*\phi_{1xxxxx}*sigma_1**2 + 54*\phi_1**4*\phi_{1xxx}*sigma_0**2*sigma_1 + 810*\phi_1**4*\phi_{1xx}*\phi_{1x}*sigma_0*sigma_1**2 + 27*\phi_1**4*\phi_{1xx}*sigma_1**2 + 540*\phi_1**4*\phi_{1x}**3*sigma_1**3 - 3*\phi_1**4*\phi_{2xx}*sigma_1**2 - 18*\phi_1**3*\phi_2*\phi_{1xx}*sigma_1**2 + 18*\phi_1**3*\phi_{1xxxxx}*sigma_0*sigma_1 + 126*\phi_1**3*\phi_{1xxxx}*\phi_{1x}*sigma_1**2 + 198*\phi_1**3*\phi_{1xxx}*\phi_{1xx}*sigma_1**2 + 18*\phi_1**3*\phi_{1xxx}*sigma_0**3 + 648*\phi_1**3*\phi_{1xx}*\phi_{1x}*sigma_0**2*sigma_1 + 45*\phi_1**3*\phi_{1xx}*sigma_0*sigma_1 + 1080*\phi_1**3*\phi_{1x}**3*sigma_0*sigma_1**2 + 108*\phi_1**3*\phi_{1x}**2*sigma_1**2 - 30*\phi_1**3*\phi_{1x}*\phi_{2x}*sigma_1**2 - 6*\phi_1**3*\phi_{2xx}*sigma_0*sigma_1 - 3*\phi_1**2*\phi_2**2*\phi_{2xx}*sigma_1**2 - 27*\phi_1**2*\phi_2*\phi_{1xx}*sigma_0*sigma_1 - 54*\phi_1**2*\phi_

h^2 =>


-972*\phi_1**9*\phi_{1xxxx}*\phi_{1x}*sigma_1**5 - 2916*\phi_1**9*\phi_{1xxx}*\phi_{1xx}*sigma_1**5 - 4374*\phi_1**8*\phi_{1xxxx}*\phi_{1x}*sigma_0*sigma_1**4 - 162*\phi_1**8*\phi_{1xxxx}*sigma_1**4 - 13122*\phi_1**8*\phi_{1xxx}*\phi_{1xx}*sigma_0*sigma_1**4 - 33372*\phi_1**8*\phi_{1xxx}*\phi_{1x}**2*sigma_1**5 - 57996*\phi_1**8*\phi_{1xx}**2*\phi_{1x}*sigma_1**5 + 162*\phi_1**7*\phi_2*\phi_{1xxxx}*sigma_1**4 - 486*\phi_1**7*\phi_{1xxxxxx}*\phi_{1x}*sigma_1**4 - 1296*\phi_1**7*\phi_{1xxxxx}*\phi_{1xx}*sigma_1**4 - 1458*\phi_1**7*\phi_{1xxxx}*\phi_{1xxx}*sigma_1**4 - 7776*\phi_1**7*\phi_{1xxxx}*\phi_{1x}*sigma_0**2*sigma_1**3 - 567*\phi_1**7*\phi_{1xxxx}*sigma_0*sigma_1**3 - 23328*\phi_1**7*\phi_{1xxx}*\phi_{1xx}*sigma_0**2*sigma_1**3 - 133488*\phi_1**7*\phi_{1xxx}*\phi_{1x}**2*sigma_0*sigma_1**4 - 6858*\phi_1**7*\phi_{1xxx}*\phi_{1x}*sigma_1**4 + 486*\phi_1**7*\phi_{1xxx}*\phi_{2x}*sigma_1**4 - 231984*\phi_1**7*\phi_{1xx}**2*\phi_{1x}*sigma_0*sigma_1**4 - 5562*\phi_1**7*\phi_{1xx}**2*s

In [15]:
F2pda = NF(F2, [f1.diff(t), f2.diff(t)], [F1, F2], head=False)
prn(F2pda)

-\phi_1 + 6*\phi_2**2*\phi_{2x}*sigma_1 + 6*\phi_2*\phi_{2x}*sigma_0 + \phi_2*sigma + \phi_2 + \phi_{2t} + \phi_{2xxx}

tau^2 =>


-3*\phi_1**4*\phi_{1xx}*sigma_1**2 - 6*\phi_1**3*\phi_{1xx}*sigma_0*sigma_1 - 12*\phi_1**3*\phi_{1x}**2*sigma_1**2 - 3*\phi_1**2*\phi_2**2*\phi_{1xx}*sigma_1**2 - 3*\phi_1**2*\phi_2*\phi_{1xx}*sigma_0*sigma_1 - 6*\phi_1**2*\phi_2*\phi_{1x}*\phi_{2x}*sigma_1**2 - 3*\phi_1**2*\phi_{1xxxx}*sigma_1/2 - 3*\phi_1**2*\phi_{1xx}*sigma_0**2 - 18*\phi_1**2*\phi_{1x}**2*sigma_0*sigma_1 - 3*\phi_1**2*\phi_{1x}*\phi_{2x}*sigma_0*sigma_1 - \phi_1**2*\phi_{1x}*sigma*sigma_1/2 - 5*\phi_1**2*\phi_{1x}*sigma_1/2 + 3*\phi_1**2*\phi_{2x}*sigma_1/2 - 18*\phi_1*\phi_2**3*\phi_{2xx}*sigma_1**2 - 3*\phi_1*\phi_2**2*\phi_{1xx}*sigma_0*sigma_1 - 6*\phi_1*\phi_2**2*\phi_{1x}**2*sigma_1**2 - 27*\phi_1*\phi_2**2*\phi_{2xx}*sigma_0*sigma_1 - 54*\phi_1*\phi_2**2*\phi_{2x}**2*sigma_1**2 - 3*\phi_1*\phi_2*\phi_{1xx}*sigma_0**2 - 6*\phi_1*\phi_2*\phi_{1x}**2*sigma_0*sigma_1 - 6*\phi_1*\phi_2*\phi_{1x}*\phi_{2x}*sigma_0*sigma_1 + 3*\phi_1*\phi_2*\phi_{1x}*sigma_1 - 3*\phi_1*\phi_2*\phi_{2xxxx}*sigma_1 - 9*\phi_1*\phi_2*

h^2 =>


54*\phi_1**7*\phi_{1xxx}*\phi_{1x}*sigma_1**4 + 54*\phi_1**7*\phi_{1xx}**2*sigma_1**4 + 189*\phi_1**6*\phi_{1xxx}*\phi_{1x}*sigma_0*sigma_1**3 + 9*\phi_1**6*\phi_{1xxx}*sigma_1**3 + 189*\phi_1**6*\phi_{1xx}**2*sigma_0*sigma_1**3 + 1080*\phi_1**6*\phi_{1xx}*\phi_{1x}**2*sigma_1**4 - 27*\phi_1**5*\phi_2*\phi_{1xxx}*sigma_1**3 + 18*\phi_1**5*\phi_{1xxxxx}*\phi_{1x}*sigma_1**3 + 27*\phi_1**5*\phi_{1xxxx}*\phi_{1xx}*sigma_1**3 + 9*\phi_1**5*\phi_{1xxx}**2*sigma_1**3 + 243*\phi_1**5*\phi_{1xxx}*\phi_{1x}*sigma_0**2*sigma_1**2 + 27*\phi_1**5*\phi_{1xxx}*sigma_0*sigma_1**2/2 + 243*\phi_1**5*\phi_{1xx}**2*sigma_0**2*sigma_1**2 + 3240*\phi_1**5*\phi_{1xx}*\phi_{1x}**2*sigma_0*sigma_1**3 + 216*\phi_1**5*\phi_{1xx}*\phi_{1x}*sigma_1**3 - 36*\phi_1**5*\phi_{1xx}*\phi_{2x}*sigma_1**3 + 1404*\phi_1**5*\phi_{1x}**4*sigma_1**4 - 9*\phi_1**5*\phi_{1x}*\phi_{2xx}*sigma_1**3 + 108*\phi_1**4*\phi_2**3*\phi_{1xxx}*\phi_{2x}*sigma_1**4 + 108*\phi_1**4*\phi_2**3*\phi_{1xx}*\phi_{2xx}*sigma_1**4 + 162*\phi_1**